In [23]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

from tensorflow import keras


In [6]:
mnist = fetch_openml('mnist_784', version = 1)
X, y = mnist['data'], mnist['target']
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = X_train.values.reshape(-1, 28, 28, 1)
X_test = X_test.values.reshape(-1, 28, 28, 1)


In [7]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [11]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

In [13]:
model_cnn = Sequential()
model_cnn.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', strides = 1, padding = 'same', input_shape = (28, 28, 1)))
model_cnn.add(BatchNormalization())
model_cnn.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model_cnn.add(Dropout(0.25))

model_cnn.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', strides = 1, padding = 'same'))
model_cnn.add(BatchNormalization())
model_cnn.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', strides = 1, padding = 'same'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten())
model_cnn.add(Dense(512, activation = 'relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(Dropout(0.25))
model_cnn.add(Dense(1024, activation = 'relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(10, activation='softmax'))



In [14]:
optimizer = Adam()
model_cnn.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics = ["accuracy"])

In [17]:
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
batch_size = 32
epochs = 30

history = model_cnn.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, callbacks = [reduce_lr, early_stopping])

Epoch 1/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9932 - loss: 0.0277 - learning_rate: 0.0010
Epoch 2/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9951 - loss: 0.0187 - learning_rate: 9.0000e-04
Epoch 3/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9951 - loss: 0.0168 - learning_rate: 8.1000e-04
Epoch 4/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9960 - loss: 0.0136 - learning_rate: 7.2900e-04
Epoch 5/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9969 - loss: 0.0123 - learning_rate: 6.5610e-04
Epoch 6/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9969 - loss: 0.0093 - learning_rate: 5.9049e-04
Epoch 7/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.9980 - loss: 0.0066 - learning_rate: 5.3144e-04
Epoch 8/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9976 - loss: 0.0089 - learning_rate: 4.7830e-04
Epoch 9/30
1750/1750 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9980 

In [18]:
model_cnn.evaluate(X_test, y_test)


438/438 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9948 - loss: 0.0221


[0.02212447300553322, 0.9954285621643066]

In [25]:
model_cnn.save('mnist_model_cnn_v3.h5')
